In [8]:
import re
import json
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

def cargar_aerolineas(archivo_csv):
    df = pd.read_csv(archivo_csv)
    return df['nombre'].str.lower().tolist()

def cargar_ciudades(archivo_csv):
    df = pd.read_csv(archivo_csv)
    return df['nombre'].str.lower().tolist()

def procesar_mensaje(mensaje, aerolineas, ciudades):
    informacion_viaje = {'origen': '', 'destino': '', 'fecha': None, 'cantidad': 1, 'aerolínea': ''}

    mensaje = mensaje.lower()

    num_palabras = {
        'uno': 1,
        'dos': 2,
        'tres': 3,
        'cuatro': 4,
        'cinco': 5,
        'seis': 6,
        'siete': 7,
        'ocho': 8,
        'nueve': 9,
        'diez': 10,
    }

    palabras_regex = '|'.join(num_palabras.keys())

    regex = re.compile(r'\b(\d+|' + palabras_regex + r')\b\s*(billetes?|boletos?|tickets?|pasajes?)')

    cantidad = regex.search(mensaje)
    if cantidad:
        numero = cantidad.group(1)
        if numero.isdigit():
            informacion_viaje['cantidad'] = int(numero)
        else:
            informacion_viaje['cantidad'] = num_palabras[numero]

    for ciudad in ciudades:
        if ciudad in mensaje:
            if not informacion_viaje['origen']:
                informacion_viaje['origen'] = ciudad.capitalize()
            elif not informacion_viaje['destino']:
                informacion_viaje['destino'] = ciudad.capitalize()

    for aerolinea in aerolineas:
        if aerolinea in mensaje:
            informacion_viaje['aerolínea'] = aerolinea.capitalize()

    fecha = re.search(r'\b(\d{1,2} de \w+)\b', mensaje)
    if fecha:
        informacion_viaje['fecha'] = fecha.group(1)

    return informacion_viaje

def on_send_message(change):
    mensaje = text_input.value.strip()
    clear_output()
    display(vbox)

    if mensaje.lower() == "salir":
        display(widgets.HTML("<b>¡Hasta luego!</b>"))
        return
    else:
        informacion_viaje = procesar_mensaje(mensaje, aerolineas, ciudades)
        if informacion_viaje['origen'] and informacion_viaje['destino']:
            result = (f"Perfecto. Comienzo la búsqueda de tu viaje de {informacion_viaje['origen']} a "
                      f"{informacion_viaje['destino']} con {informacion_viaje['aerolínea']}.\n")
            if informacion_viaje['fecha']:
                result += f"Fecha: {informacion_viaje['fecha']}\n"
            result += ("Información del viaje:\n"
                       f"Origen: {informacion_viaje['origen']}\n"
                       f"Destino: {informacion_viaje['destino']}\n"
                       f"Cantidad: {informacion_viaje['cantidad']}\n"
                       f"Aerolínea: {informacion_viaje['aerolínea']}\n")
            if informacion_viaje['fecha']:
                result += f"Fecha: {informacion_viaje['fecha']}\n"
        else:
            result = "No entendí tu solicitud. Por favor intenta de nuevo.\n"
            result += "No se encontró información del viaje solicitado."

        json_result = json.dumps(informacion_viaje, indent=4, ensure_ascii=False)
        output.append_display_data(widgets.HTML(f"<b>Usuario:</b> {mensaje}<br><b>Asistente:</b> {result}<br><pre>{json_result}</pre>"))
        text_input.value = ""

aerolineas = cargar_aerolineas('aerolineas.csv')
ciudades = cargar_ciudades('ciudades.csv')

text_input = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output = widgets.Output()

send_button.on_click(on_send_message)

vbox = widgets.VBox([widgets.HTML("<b>Hola, bienvenido a Sky2travel. ¿Cómo te puedo ayudar?</b>"), text_input, send_button, output])
display(vbox)


HTML(value='<b>Usuario:</b> deseo  7 boletos de avion para salir de bogota a tokio para el 15 de julio con avi…